In [12]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
trainData = pd.read_csv("./titanikData.csv")
testData = pd.read_csv("./titanikTest.csv", names=["pclass", "age", "gender", "survived"])

In [13]:
number = LabelEncoder()
trainData['survived'] = number.fit_transform(trainData['survived'])
trainData['gender'] = number.fit_transform(trainData['gender'])
trainData['age'] = number.fit_transform(trainData['age'])
trainData['pclass'] = number.fit_transform(trainData['pclass'])
trainData.head()

,pclass,age,gender,survived
0,0,0,1,1
1,0,0,1,1
2,0,0,1,1
3,0,0,1,1
4,0,0,1,1


In [15]:
testData['survived'] = number.fit_transform(testData['survived'])
testData['gender'] = number.fit_transform(testData['gender'])
testData['age'] = number.fit_transform(testData['age'])
testData['pclass'] = number.fit_transform(testData['pclass'])
testData.head()

,pclass,age,gender,survived
0,0,0,1,1
1,0,0,1,1
2,0,0,1,1
3,0,0,1,1
4,0,0,1,1


In [16]:
trees = 100 #Initialize
tree_models = []
algorithm_samples = []

In [17]:
unique_rows = trainData.drop_duplicates()
for i in range(trees):
    samples = unique_rows.sample(frac = 0.65)
    complementary_size = len(trainData) - len(samples)
    complement = samples.sample(n = complementary_size, replace = True)
    algorithm_samples.append(pd.concat([samples, complement]))


In [19]:
from sklearn.tree import DecisionTreeClassifier
for temp in range(trees): # training the model by 'survived'
    features_train = algorithm_samples[temp].drop(['survived'], axis = 1)
    target_train = algorithm_samples[temp]['survived'].where(algorithm_samples[temp]['survived'] == 1, 0)
    tree_model = DecisionTreeClassifier(criterion = "entropy", max_depth = 1)
    model = tree_model.fit(features_train, target_train)
    tree_models.append(model)

In [22]:
# predicting the model by 'survived'
predictions = []
for i, model in enumerate(tree_models):
    predictions.append(model.predict(testData.drop(['survived'], axis=1)))
pred_matrix = np.array(predictions)
pred_matrix

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [30]:
u, c = np.unique(pred_matrix, return_counts=True)
result = u[c.argmax()]
# checking results
testData['predictions'] = result
testData['corrects_results'] = (result == testData.survived)
correct_results = np.sum(testData['corrects_results'].astype(int))
print("Success rates are - ", (correct_results / len(testData.predictions)) * 100, "%")

Success rates are -  92.42424242424242 %


In [31]:
new_test_data = pd.read_csv("./titanikTest.csv", names=["pclass", "age", "gender", "survived"])
new_test_data['predictions'] = np.where(testData['predictions'] == 1, 'yes', 'no')
print(new_test_data)

   pclass    age gender survived predictions
0     1st  adult   male      yes         yes
1     1st  adult   male      yes         yes
2     1st  adult   male      yes         yes
3     1st  adult   male      yes         yes
4     1st  adult   male      yes         yes
..    ...    ...    ...      ...         ...
61    3rd  adult   male      yes         yes
62    3rd  adult   male      yes         yes
63    3rd  adult   male      yes         yes
64    3rd  adult   male      yes         yes
65    3rd  adult   male      yes         yes

[66 rows x 5 columns]
